In [22]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame, read_csv

from ML.DTGenerated import DTGenerated
import pickle

In [23]:
from generateTestingData.useGeneratedData import makeDataUsable, splitTrainTest, NewPredicter

In [24]:
MODEL_PREFIX = "gen"

In [25]:
df = makeDataUsable(read_csv("../data/generatedData.csv"),4, 3)
print(df.shape)
df.head()

(3950, 30)


,Time,TTC1,TTC2,TTC3,TTC4,DTO1,DTO2,DTO3,DTO4,JERK1,...,asX4,asY1,asY2,asY3,asY4,asZ1,asZ2,asZ3,asZ4,COL
0,3.0,50.0,50.0,50.0,50.000,16.6264,100.0,100.0,100.0000,0.004,...,0.000,0.000,-0.000,-0.001,-0.001,0.000,0.006,0.001,0.000,0.0
1,4.0,50.0,50.0,50.0,50.000,100.0000,100.0,100.0,100.0000,0.661,...,-0.001,-0.000,-0.001,-0.001,0.000,0.006,0.001,0.000,-0.001,0.0
2,5.0,50.0,50.0,50.0,50.000,100.0000,100.0,100.0,100.0000,4.117,...,-0.000,-0.001,-0.001,0.000,0.000,0.001,0.000,-0.001,0.000,0.0
3,6.0,50.0,50.0,50.0,50.000,100.0000,100.0,100.0,100.0000,0.865,...,0.000,-0.001,0.000,0.000,0.000,0.000,-0.001,0.000,0.000,0.0
4,7.0,50.0,50.0,50.0,5.322,100.0000,100.0,100.0,83.3296,3.704,...,0.004,0.000,0.000,0.000,-0.001,-0.001,0.000,0.000,0.005,0.0


In [26]:
df["COL"].value_counts()

0.0    3696
1.0     254
Name: COL, dtype: int64

In [27]:

# sampleSize = int(len(data[data['COL'] == 1]) * undersampleRatio)
# print("Undersampling")
# data = concat([data[data["COL"] == 0].sample(sampleSize, random_state=1), data[data["COL"] == True]])

TODO: need to either get more crash data or keep undersample below

In [28]:
df = pd.concat([df[df["COL"] == False].sample(200, random_state=1), df[df["COL"] == True]])

In [29]:
data = df.copy().drop(columns=df.columns[0], axis=1)
trainX, trainY, testX, testY = splitTrainTest(data)

Splitting at 363, total rows: 454


In [30]:
trainX.head(3)

,TTC1,TTC2,TTC3,TTC4,DTO1,DTO2,DTO3,DTO4,JERK1,JERK2,...,asX3,asX4,asY1,asY2,asY3,asY4,asZ1,asZ2,asZ3,asZ4
2983,50.000,50.000,50.000,50.000,100.0000,100.000,100.0000,100.000,2.200,3.084,...,0.003,0.004,-0.638,-0.001,-0.0,0.022,0.018,-0.02,0.002,0.008
892,50.000,1.143,50.000,50.000,8.5427,4.892,9.7264,14.780,6.825,5.762,...,-0.009,-0.004,-0.000,0.000,-0.0,-0.001,0.001,-0.01,0.009,0.004
3106,2.426,2.004,1.608,0.764,14.1148,11.342,8.6948,5.289,0.016,0.016,...,-0.000,-0.000,0.000,0.000,0.0,0.000,-0.000,-0.00,-0.000,-0.000


## Model

In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

### MLP Classifier

In [32]:
mlp_clf = NewPredicter()
mlp_trainX = mlp_clf.preProcess(trainX.copy())
mlp_testX = mlp_clf.preProcess(testX.copy())

mlp_trainY, mlp_testY = trainY.to_numpy(), testY.to_numpy()

mlp_clf.fit(mlp_trainX, mlp_trainY)

Scaler is fitted


In [33]:
mlp_pred = mlp_clf.predict(mlp_testX)
mlp_score = mlp_clf.getScore(mlp_pred, mlp_testY)

Total: 91, Collisions: 53
Accuracy: 0.6923076923076923, Precision: 0.7358490566037735, Recall: 0.7358490566037735, F1: 0.7358490566037735
Confusion matrix:
[[24 14]
 [14 39]]


In [34]:
model_score = f"{mlp_score[0][0]}-{mlp_score[0][1]}-{mlp_score[1][0]}-{mlp_score[1][1]}"
mlp_clf.saveModel(f"MLPClassifier_{MODEL_PREFIX}", accuracy=model_score)

Model saved!


### Random Forest Classifier

In [35]:
rf_clf = NewPredicter(RandomForestClassifier(n_estimators=100, max_depth=10, random_state=1))
rf_trainX = rf_clf.preProcess(trainX.copy())
rf_testX = rf_clf.preProcess(testX.copy())

rf_trainY, rf_testY = trainY.to_numpy(), testY.to_numpy()

rf_clf.fit(rf_trainX, rf_trainY)

Scaler is fitted


In [36]:
rf_pred = rf_clf.predict(rf_testX)
rf_score = rf_clf.getScore(rf_pred, rf_testY)

Total: 91, Collisions: 50
Accuracy: 0.7692307692307693, Precision: 0.7735849056603774, Recall: 0.82, F1: 0.796116504854369
Confusion matrix:
[[29 12]
 [ 9 41]]


In [37]:
model_score = f"{rf_score[0][0]}-{rf_score[0][1]}-{rf_score[1][0]}-{rf_score[1][1]}"
rf_clf.saveModel(f"RandomForestClassifier_{MODEL_PREFIX}", accuracy=model_score)

Model saved!


### SVM Classifier

In [38]:
svm_clf = NewPredicter(SVC(random_state=1))
svm_trainX = svm_clf.preProcess(trainX.copy())
svm_testX = svm_clf.preProcess(testX.copy())

svm_trainY, svm_testY = trainY.to_numpy(), testY.to_numpy()

svm_clf.fit(svm_trainX, svm_trainY)

Scaler is fitted


In [39]:
svm_pred = svm_clf.predict(svm_testX)
svm_score = svm_clf.getScore(svm_pred, svm_testY)

Total: 91, Collisions: 50
Accuracy: 0.7032967032967034, Precision: 0.7169811320754716, Recall: 0.76, F1: 0.7378640776699028
Confusion matrix:
[[26 15]
 [12 38]]


In [40]:
model_score = f"{svm_score[0][0]}-{svm_score[0][1]}-{svm_score[1][0]}-{svm_score[1][1]}"
svm_clf.saveModel(f"SVC_{MODEL_PREFIX}", accuracy=model_score)

Model saved!


### XGBoost Classifier

In [41]:
xgb_clf = NewPredicter(XGBClassifier(n_estimators=100, random_state=1))
xgb_trainX = xgb_clf.preProcess(trainX.copy())
xgb_testX = xgb_clf.preProcess(testX.copy())

xgb_trainY, xgb_testY = trainY.to_numpy(), testY.to_numpy()

Scaler is fitted


In [42]:
xgb_clf.fit(xgb_trainX, xgb_trainY)

In [43]:
len(xgb_testX[0])
xgb_pred = xgb_clf.predict(xgb_testX)
xgb_score = xgb_clf.getScore(xgb_testY, xgb_pred)

Total: 91, Collisions: 53
Accuracy: 0.7692307692307693, Precision: 0.7962962962962963, Recall: 0.8113207547169812, F1: 0.8037383177570094
Confusion matrix:
[[27 11]
 [10 43]]


TODO - need more data, what is radar?

In [44]:
# model_score = f"{score[0][0]}-{score[0][1]}-{score[1][0]}-{score[1][1]}"
model_score = f"{xgb_score[0][0]}-{xgb_score[0][1]}-{xgb_score[1][0]}-{xgb_score[1][1]}"
xgb_clf.saveModel(f"XGBClassifier_{MODEL_PREFIX}", accuracy=model_score)

Model saved!
